# M1M3 hardpoint learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')


In [ ]:
# Times of breakaway test
start = "2024-10-11T15:00:00"
end = "2024-10-12T00:30:00"

In [ ]:
hardpoints = await client.select_time_series("lsst.sal.MTM1M3.logevent_hardpointTestStatus", 
                                             "*", Time(start, scale='utc'), Time(end, scale='utc'))
len(hardpoints)

In [ ]:
hardpoints.columns

## class HardpointTest(enum.IntEnum): 
    NOTTESTED = 1 
    MOVINGNEGATIVE = 2 
    TESTINGPOSITIVE = 3 
    TESTINGNEGATIVE = 4 
    MOVINGREFERENCE = 5 
    PASSED = 6 
    FAILED = 7 

In [ ]:
for i in range(6):
    column = f"testState{i}"
    hardpoints[column].plot()

## First look at the forces and displacements during the hardpoint breakaway test

In [ ]:
nx = 3; ny = 2
fig, axs = plt.subplots(nx, ny, figsize=(8,10))
plt.subplots_adjust(hspace=0.7, wspace=0.7)
timestamp = hardpoints.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.suptitle(f"Hardpoints tests {timestamp}")
for i in range(nx):
    for j in range(ny):
        try:
            ax = axs[i,j]
            index = i * ny + j
            plotStart = hardpoints[hardpoints[f'testState{index}']==2]['private_kafkaStamp'][0] - 1.0
            plotEnd = hardpoints[hardpoints[f'testState{index}']==6]['private_kafkaStamp'][0] + 1.0
            testingPositive = hardpoints[hardpoints[f'testState{index}']==3]['private_kafkaStamp'][0]
            testingPositive = Time(testingPositive, format='unix_tai', scale='tai').utc.isot
            testingNegative = hardpoints[hardpoints[f'testState{index}']==4]['private_kafkaStamp'][0]
            testingNegative = Time(testingNegative, format='unix_tai', scale='tai').utc.isot
            movRef = hardpoints[hardpoints[f'testState{index}']==5]['private_kafkaStamp'][0]
            movRef = Time(movRef, format='unix_tai', scale='tai').utc.isot
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start.utc, end.utc)
            imsData = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start.utc, end.utc)
            hardpointData[f'displacement{index}'].plot(ax=ax, color='green', label='Displacement')
            ax.set_ylim(-0.01,0.025)
            ax.axvline(testingPositive, ls='--')
            ax.axvline(testingNegative, ls='--')
            ax.axvline(movRef, ls='--')
            ax2 = ax.twinx()
            hardpointData[f'measuredForce{index}'].plot(ax=ax2, color='red', label='Force')
            ax.set_title(f'Hardpoint {index}')
            ax.set_ylabel('Displacement (m)')
            ax.legend(loc='upper left', fontsize=6)
            ax2.legend(loc='upper right', fontsize=6)
            ax2.set_ylim(-4000, 4000)
            ax2.set_ylabel('Force (N)')
        except:
            continue
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Test{timestamp}.pdf")

## Look at the hardpoint displacements

In [ ]:
plt.subplot(2,1,1)
hardpointData['xPosition'].plot(label='X')
hardpointData['yPosition'].plot(label='Y')
hardpointData['zPosition'].plot(label='Z')
plt.legend()
plt.subplot(2,1,2)
hardpointData['xRotation'].plot(label='XRot')
hardpointData['yRotation'].plot(label='YRot')
hardpointData['zRotation'].plot(label='ZRot')
plt.legend()

In [ ]:
plt.subplot(2,1,1)
imsData['xPosition'].plot(label='X')
imsData['yPosition'].plot(label='Y')
imsData['zPosition'].plot(label='Z')
plt.legend()
plt.subplot(2,1,2)
imsData['xRotation'].plot(label='XRot')
imsData['yRotation'].plot(label='YRot')
imsData['zRotation'].plot(label='ZRot')
plt.legend()

In [ ]:
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position IMS vs HP during HP breakaway test", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        
        smoothed_imsData = imsData[plots[plot_counter]].rolling(10).mean()
        smoothed_imsData = smoothed_imsData.dropna()
        smoothed_imsData -= smoothed_imsData[0]
        smoothed_hardpointData = hardpointData[plots[plot_counter]].rolling(10).mean()
        smoothed_hardpointData = smoothed_hardpointData.dropna()
        smoothed_hardpointData -= smoothed_hardpointData[0]
        
        if plot_counter > 2:
            smoothed_imsData *= 1E6
            smoothed_hardpointData *= 1E6
            unit = 'microDeg'
        else:
            smoothed_imsData *= 1E6
            smoothed_hardpointData *= 1E6
            unit = 'um'

        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'HP Displacement({unit})')
        axs[i][j].yaxis.label.set_color('blue')
        ax = axs[i][j].twinx()
        ax.set_ylabel(f'IMS ({unit})')
        ax.yaxis.label.set_color('red')
        #initial_pos = np.median(smoothed_ims_data.values[0:100])
        #final_pos = np.median(smoothed_ims_data.values[-100:-1])
        #pos_delta = initial_pos - final_pos
        #axs[i][j].set_title(plots[plot_counter]+f"\n Position Delta = {pos_delta:.1f} {unit}")

        smoothed_hardpointData.plot(ax=axs[i][j], color='blue', label='Hardpoint')
        smoothed_imsData.plot(ax=ax, color='red', label='IMS')
        plot_counter += 1


## Now blow up the region where the force changes rapidly

In [ ]:
def stiffness(force, disp):
    forces = force.values
    disps = disp.values
    foundMin = False
    foundMax = False
    for i in range(len(forces)):
        #print(i, forces[i])
        if forces[i] < 1000.0 and not foundMax:
            forceMax = forces[i]
            dispMax = disps[i] * 1.0E6
            foundMax = True
        if forces[i] < -1000.0 and not foundMin:
            forceMin = forces[i]
            dispMin = disps[i] * 1.0E6
            foundMin = True
            break
    print(forceMin, forceMax, dispMin, dispMax)
    stiffness = (forceMax - forceMin) / (dispMax - dispMin)
    print(f"Stiffness = {stiffness:.2f} N/microns")
    return stiffness

def stiffnessValues(forceValues, dispValues):
    forces = forceValues
    disps = dispValues
    foundMin = False
    foundMax = False
    for i in range(len(forces)):
        #print(i, forces[i])
        if forces[i] < 1000.0 and not foundMax:
            forceMax = forces[i]
            dispMax = disps[i]
            foundMax = True
        if forces[i] < -1000.0 and not foundMin:
            forceMin = forces[i]
            dispMin = disps[i]
            foundMin = True
            break
    #print(forceMin, forceMax, dispMin, dispMax)
    stiffness = (dispMax - dispMin) / (forceMax - forceMin)
    #print(f"Stiffness = {stiffness:.2f} microns/N")
    return stiffness

In [ ]:
len(force.values)

In [ ]:
stiffness(force, disp)

In [ ]:
states = [['Pos',3,4], ['Neg',4,5]]
nx = 3; ny = 2
tPlot = 15.0
fig, axs = plt.subplots(nx, ny, figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
timestamp = hardpoints.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.suptitle(f"Hardpoints tests {timestamp}", fontsize=24)
for i in range(nx):
    for j in range(ny):
        ax = axs[i,j]
        index = i * ny + j
        for [name,startState, endState] in states:
            getStart = hardpoints[hardpoints[f'testState{index}']==startState]['private_kafkaStamp'][0]
            getEnd = hardpoints[hardpoints[f'testState{index}']==endState]['private_kafkaStamp'][0]
            start = Time(getStart, format='unix_tai', scale='tai')
            end = Time(getEnd, format='unix_tai', scale='tai')
            hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                            [f'measuredForce{index}', 'timestamp'], start.utc, end.utc)
            plotStart = hardpointData.loc[abs(hardpointData[f'measuredForce{index}']) < 100.0]['timestamp'][-1] - tPlot/2.0
            plotEnd = plotStart + tPlot
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                            [f'displacement{index}', f'measuredForce{index}'], start.utc, end.utc)
            force = hardpointData[f'measuredForce{index}']
            disp = hardpointData[f'displacement{index}']
            
            ax.plot(disp, force, label=name)
            ax.set_title(f'Hardpoint {index}')
            ax.set_ylabel('Force (N)')
            ax.set_ylim(-4000, 4000)
            ax.set_xlabel('Displacement(m)')
        stiff = stiffness(force, disp)
        ax.text(disp.values.mean(), -3000, f"Stiffness = \n{stiff:.1f} microns/N")
        ax.legend(loc='upper left')
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Test_Blowup_{timestamp}.pdf")

In [ ]:
posNames = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
timestamp = hardpoints.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/HP_Breakaway_Test_Impact_on_IMS_{timestamp}.pdf")
for n, posName in enumerate(posNames):
    if n > 2:
        unit = 'microDeg'
    else:
        unit = 'um'

    states = [['Pos',3,4], ['Neg',4,5]]
    nx = 3; ny = 2
    tPlot = 15.0
    fig, axs = plt.subplots(nx, ny, figsize=(8,10))
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    plt.suptitle(f"Impact of hardpoint breakaway tests on {posName}")
    for i in range(nx):
        for j in range(ny):
            ax = axs[i,j]
            index = i * ny + j
            for [name,startState, endState] in states:
                getStart = hardpoints[hardpoints[f'testState{index}']==startState]['private_kafkaStamp'][0]
                getEnd = hardpoints[hardpoints[f'testState{index}']==endState]['private_kafkaStamp'][0]
                start = Time(getStart, format='unix_tai', scale='tai')
                end = Time(getEnd, format='unix_tai', scale='tai')
                hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                                [f'measuredForce{index}', 'timestamp'], start.utc, end.utc)
                plotStart = hardpointData.loc[abs(hardpointData[f'measuredForce{index}']) < 100.0]['timestamp'][-1] - tPlot/2.0
                plotEnd = plotStart + tPlot
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                                [f'displacement{index}', f'measuredForce{index}', 'timestamp'], start.utc, end.utc)
                imsData = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start.utc, end.utc)
                force = hardpointData[f'measuredForce{index}']
                forceData = force.values
                forceTimes = hardpointData['timestamp'].values
                dispData = imsData[posName].values * 1.0E6
                dispTimes = imsData['timestamp'].values
                disp = np.interp(forceTimes, dispTimes, dispData)

                ax.plot(disp, forceData, label=name)
                ax.set_title(f'Hardpoint {index}')
                ax.set_ylabel('Force (N)')
                ax.set_ylim(-4000, 4000)
                ax.set_xlabel(f'{posName} ({unit})')
            stiff = stiffnessValues(force, disp)
            ax.text(dispData.mean(), -3000, f"Stiffness = \n{stiff:.4f} {unit}/N")
            ax.legend(loc='upper left')
    pdf.savefig(fig) 
    plt.clf()
pdf.close()
